In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import networkx as nx

import torch
import dgl
from tqdm import tqdm
from qtaim_embed.utils.grapher import get_grapher
from qtaim_embed.data.molwrapper import mol_wrappers_from_df
from qtaim_embed.utils.tests import get_data
from qtaim_embed.core.dataset import HeteroGraphNodeLabelDataset

In [2]:
import pandas as pd 
#import pymatgen
#import pickle5 as pickle
#with open("/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl", "rb") as fh:
#    df = pickle.load(fh)
df = pd.read_pickle("/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl")
                    

In [3]:
df.keys()

Index(['molecule', 'molecule_graph', 'bonds', 'ids', 'names',
       'extra_feat_atom_Lagrangian_K', 'extra_feat_atom_Hamiltonian_K',
       'extra_feat_atom_e_density', 'extra_feat_atom_lap_e_density',
       'extra_feat_atom_e_loc_func', 'extra_feat_atom_ave_loc_ion_E',
       'extra_feat_atom_delta_g_promolecular', 'extra_feat_atom_delta_g_hirsh',
       'extra_feat_atom_esp_nuc', 'extra_feat_atom_esp_e',
       'extra_feat_atom_esp_total', 'extra_feat_atom_grad_norm',
       'extra_feat_atom_lap_norm', 'extra_feat_atom_eig_hess',
       'extra_feat_atom_det_hessian', 'extra_feat_atom_ellip_e_dens',
       'extra_feat_atom_eta', 'extra_feat_bond_Lagrangian_K',
       'extra_feat_bond_Hamiltonian_K', 'extra_feat_bond_e_density',
       'extra_feat_bond_lap_e_density', 'extra_feat_bond_e_loc_func',
       'extra_feat_bond_ave_loc_ion_E', 'extra_feat_bond_delta_g_promolecular',
       'extra_feat_bond_delta_g_hirsh', 'extra_feat_bond_esp_nuc',
       'extra_feat_bond_esp_e', 'extra_fea

In [4]:
train_dataset = HeteroGraphNodeLabelDataset(
    #file="/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_full.pkl",
    file="/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl",
    allowed_ring_size=[3, 4, 5, 6, 7],
    allowed_charges=None,
    self_loop=True,
    extra_keys={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": [
            "extra_feat_bond_esp_total",
            'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta',
            "bond_length",
        ],
        "global": [],
    },
    target_dict={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": ["extra_feat_bond_esp_total", 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta',],
    },
    extra_dataset_info={},
    debug=True,
    log_scale_targets=False,
    standard_scale_targets=True,
)

... > running in debug mode
... > creating MoleculeWrapper objects


100%|██████████| 100/100 [00:00<00:00, 3058.79it/s]


element set {'N', 'O', 'C', 'H'}
selected atomic keys ['extra_feat_atom_esp_total']
selected bond keys ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta', 'bond_length']
selected global keys []
... > Building graphs and featurizing


100%|██████████| 100/100 [00:00<00:00, 123.59it/s]


included in labels
{'atom': ['extra_feat_atom_esp_total'], 'bond': ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta'], 'global': []}
included in graph features
{'atom': ['total_degree', 'total_H', 'is_in_ring', 'ring_size_3', 'ring_size_4', 'ring_size_5', 'ring_size_6', 'ring_size_7', 'chemical_symbol_N', 'chemical_symbol_O', 'chemical_symbol_C', 'chemical_symbol_H'], 'bond': ['metal bond', 'ring inclusion', 'ring size_3', 'ring size_4', 'ring size_5', 'ring size_6', 'ring size_7', 'bond_length'], 'global': ['num atoms', 'num bonds', 'molecule weight']}
... > parsing labels and features in graphs


100%|██████████| 100/100 [00:00<00:00, 3840.97it/s]


... > Scaling features
Standard deviation for feature 0 is 0.0, smaller than 0.001. You may want to exclude this feature.
... > Scaling features complete
... > mean: 
 {'atom': tensor([2.0438, 0.5204, 0.2786, 0.0332, 0.0576, 0.0983, 0.0620, 0.0276, 0.0657,
        0.0802, 0.3413, 0.5128]), 'bond': tensor([0.0000, 0.2825, 0.0349, 0.0637, 0.1011, 0.0662, 0.0300, 1.2698]), 'global': tensor([ 15.9700,  16.3200, 108.9015])}
... > std:  
 {'atom': tensor([1.2534, 0.8735, 0.4483, 0.1791, 0.2330, 0.2977, 0.2411, 0.1637, 0.2478,
        0.2715, 0.4741, 0.4998]), 'bond': tensor([0.0010, 0.4502, 0.1836, 0.2443, 0.3015, 0.2486, 0.1707, 0.2240]), 'global': tensor([2.6700, 2.8562, 6.5972])}
... > Scaling targets
... > Scaling targets complete
... > mean: 
 {'atom': tensor([2084470.5000]), 'bond': tensor([0.9842, 0.0764, 1.4701])}
... > std:  
 {'atom': tensor([5203774.5000]), 'bond': tensor([0.4207, 0.1651, 0.4141])}
... > loaded dataset


In [5]:
# TODO: build dataloader class
from qtaim_embed.core.dataset import HeteroGraphNodeLabelDataset, Subset

test_subset = Subset(train_dataset, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
test_subset.feature_names()

{'atom': ['total_degree',
  'total_H',
  'is_in_ring',
  'ring_size_3',
  'ring_size_4',
  'ring_size_5',
  'ring_size_6',
  'ring_size_7',
  'chemical_symbol_N',
  'chemical_symbol_O',
  'chemical_symbol_C',
  'chemical_symbol_H'],
 'bond': ['metal bond',
  'ring inclusion',
  'ring size_3',
  'ring size_4',
  'ring size_5',
  'ring size_6',
  'ring size_7',
  'bond_length'],
 'global': ['num atoms', 'num bonds', 'molecule weight']}

In [6]:
len_dict = {}
for key, value in test_subset.dataset.exclude_names.items():
    len_dict[key] = len(value)
len_dict

{'atom': 12, 'bond': 8, 'global': 3}

In [7]:
from qtaim_embed.data.dataloader import DataLoaderMoleculeNodeTask

In [10]:
dataloader = DataLoaderMoleculeNodeTask(train_dataset, batch_size=100, shuffle=True)

In [11]:
batch_graph, batch_label = next(iter(dataloader))

In [12]:
import dgl.nn.pytorch as dglnn

len_dict = train_dataset.featuze_size()
atom_input_size = len_dict["atom"]
bond_input_size = len_dict["bond"]
global_input_size = len_dict["global"]
bond_output_size = 3
print(atom_input_size, bond_input_size, global_input_size)

from dgl import apply_each
import torch.nn.functional as F
from torch import nn
from qtaim_embed.models.layers import GraphConvDropoutBatch


n_heads = 1
hidden_feats = 64

class testmodel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = dglnn.HeteroGraphConv(
            {
                "a2b": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=bond_input_size,
                    dropout=0.2
                ),
                "b2a": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=atom_input_size,
                    dropout=0.2
                ),
                "a2g": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=global_input_size,
                    dropout=0.2
                ),
                "g2a": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=atom_input_size,
                    dropout=0.2
                ),
                "b2g": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=global_input_size,
                    dropout=0.2
                ),
                "g2b": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=bond_input_size,
                    dropout=0.2
                ),
                "a2a": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=atom_input_size,
                    dropout=0.2
                ),
                "b2b": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=bond_input_size,
                    dropout=0.2
                ),
                "g2g": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=global_input_size,
                    dropout=0.2
                ),
            },
            aggregate="sum",
        )

        self.conv2 = dglnn.HeteroGraphConv(
            {
                "a2b": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=bond_output_size,
                ),
                "b2a": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=atom_input_size,
                ),
                "a2g": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=global_input_size,
                ),
                "g2a": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=atom_input_size,
                ),
                "b2g": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=global_input_size,
                ),
                "g2b": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=bond_output_size,
                ),
                "a2a": GraphConvDropoutBatch(
                    in_feats=atom_input_size,
                    out_feats=atom_input_size,
                ),
                "b2b": GraphConvDropoutBatch(
                    in_feats=bond_input_size,
                    out_feats=bond_output_size,
                ),
                "g2g": GraphConvDropoutBatch(
                    in_feats=global_input_size,
                    out_feats=global_input_size,
                ),
            },
            aggregate="sum",
        )
        """
        # get max feature length 
        self.max_feature_len = max([atom_input_size, bond_input_size, global_input_size])
        
        self.conv3 = dglnn.HeteroGraphConv(
            {
                "b2a": dglnn.GATConv(
                    in_feats=self.max_feature_len, 
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "g2a": dglnn.GATConv(
                    in_feats=self.max_feature_len, 
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "a2a": dglnn.GATConv(
                    in_feats=self.max_feature_len, 
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "g2b": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "a2b": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats ,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "b2b": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),

                "b2g": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "g2g": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats=hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
                "a2g": dglnn.GATConv(
                    in_feats=self.max_feature_len,
                    out_feats= hidden_feats,
                    num_heads = n_heads, 
                    #aggregator_type="lstm",
                ),
            },
            aggregate="sum",
        )

        self.conv4 = dglnn.HeteroGraphConv(
            {
                "b2a": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads, 
                    out_feats=atom_input_size,
                    #aggregator_type="lstm",
                ),
                "g2a": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads, 
                    out_feats=atom_input_size,
                    #aggregator_type="lstm",
                ),
                "a2a": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads, 
                    out_feats=atom_input_size,
                    #aggregator_type="lstm",
                ),
                "g2b": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=bond_output_size,
                    #aggregator_type="lstm",
                ),
                "a2b": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=bond_output_size,
                    #aggregator_type="lstm",
                ),
                "b2b": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=bond_output_size,
                    #aggregator_type="lstm",
                ),

                "b2g": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=global_input_size,
                    #aggregator_type="lstm",
                ),
                "g2g": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats=global_input_size,
                    #aggregator_type="lstm",
                ),
                "a2g": dglnn.GraphConv(
                    in_feats=hidden_feats * n_heads,
                    out_feats= global_input_size,
                ),
            },
            aggregate="sum",
        )
        """
    def forward(self, graph, inputs):

        """
        # feats = self.conv(graph, inputs)
        # feats = self.conv2(graph, feats)
        #print(inputs.keys())
        #for i in inputs.keys():
        #    print(i)
        #    print(inputs[i].shape)
        # zero pad with typed linear to get max feature length for each feature 
        inputs = {k: F.pad(v, (0, self.max_feature_len - v.shape[1])) for k, v in inputs.items()}
        feats = self.conv3(graph, inputs)
        #print("pass" * 30)
        feats = {k: torch.reshape(F.relu(v), (v.shape[0], -1)) for k, v in feats.items()}
        feats = self.conv4(graph, feats)
        return feats
        """

        #print(inputs)
        feats = self.conv(graph, inputs)
        #print(feats)
        feats = self.conv2(graph, feats)
        return feats



model = testmodel()

12 8 3


In [13]:
graph = batch_graph
forward_out = model(graph, graph.ndata["feat"])

In [14]:
graph = batch_graph
forward_out = model(graph, graph.ndata["feat"])

from qtaim_embed.models.node_level.base_gnn import GCNNodePred 

model_imported = GCNNodePred(
    atom_input_size=atom_input_size,
    bond_input_size=bond_input_size,
    global_input_size=global_input_size,
    n_conv_layers=2,
    target_dict={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": ["extra_feat_bond_esp_total", 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta',],
    },
    dropout=0.2,
)

In [15]:
dataloader = DataLoaderMoleculeNodeTask(train_dataset, batch_size=100, shuffle=True)
batch_graph, batch_label = next(iter(dataloader))
model.forward(batch_graph, batch_label)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1597x1 and 12x12)

In [14]:
from torch.nn import functional as F
from sklearn.metrics import r2_score

# from tqdm import tqdm
import tqdm.notebook as tq

opt = torch.optim.Adam(model.parameters(), lr=0.01)


for epoch in range(50):
    training_loss_list = []
    with tqdm(dataloader) as tq:
        model_imported.train()
        r2_list = []
        tq.set_description(f"Epoch {epoch+1}")
        training_loss = 0
        target_type = "bond"
        for step, (batch_graph, batch_label) in enumerate(tq):
            # forward propagation by using all nodes and extracting the user embeddings
            batch_graph, batch_label = next(iter(dataloader))
            labels = batch_label[target_type]
            logits = model_imported(batch_graph, batch_graph.ndata["feat"])[target_type]
            #print(logits.shape)
            #print(labels.shape)
            # compute loss
            loss = F.mse_loss(logits, labels)
            training_loss_list.append(loss.item())
            # loss_mae = F.l1_loss(logits, labels)
            # compute r2 score
            r2 = r2_score(logits.detach().numpy(), labels.detach().numpy())
            r2_list.append(r2)
            # Compute validation accuracy.  Omitted in this example.
            # backward propagation
            opt.zero_grad()
            loss.backward()
            opt.step()
            training_loss += loss.item()
            # tq.set_postfix({"Step": step, "MSE": loss.item()})

        r2_mean = np.mean(r2_list)
        loss = np.mean(training_loss_list)
        tq.set_postfix({"final_t_loss": training_loss, "R_2": r2_mean})
        print(r2_mean, loss)

        # tq.update()
        tq.close()

Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s]


TypeError: new(): invalid data type 'str'

In [43]:
from torch.nn import functional as F
from sklearn.metrics import r2_score

# from tqdm import tqdm
import tqdm.notebook as tq

opt = torch.optim.Adam(model.parameters(), lr=0.01)


for epoch in range(50):
    training_loss_list = []
    with tqdm(dataloader) as tq:
        model.train()
        r2_list = []
        tq.set_description(f"Epoch {epoch+1}")
        training_loss = 0
        target_type = "bond"
        for step, (batch_graph, batch_label) in enumerate(tq):
            # forward propagation by using all nodes and extracting the user embeddings
            batch_graph, batch_label = next(iter(dataloader))
            labels = batch_label[target_type]
            logits = model(batch_graph, batch_graph.ndata["feat"])[target_type]
            #print(logits.shape)
            #print(labels.shape)
            # compute loss
            loss = F.mse_loss(logits, labels)
            training_loss_list.append(loss.item())
            # loss_mae = F.l1_loss(logits, labels)
            # compute r2 score
            r2 = r2_score(logits.detach().numpy(), labels.detach().numpy())
            r2_list.append(r2)
            # Compute validation accuracy.  Omitted in this example.
            # backward propagation
            opt.zero_grad()
            loss.backward()
            opt.step()
            training_loss += loss.item()
            # tq.set_postfix({"Step": step, "MSE": loss.item()})

        r2_mean = np.mean(r2_list)
        loss = np.mean(training_loss_list)
        tq.set_postfix({"final_t_loss": training_loss, "R_2": r2_mean})
        print(r2_mean, loss)

        # tq.update()
        tq.close()

Epoch 1: 100%|██████████| 1/1 [00:00<00:00, 23.45it/s]


-3.7205302931984163 14.68019962310791


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 31.44it/s]


-3.9273690115274005 14.689446449279785


Epoch 3: 100%|██████████| 1/1 [00:00<00:00, 21.65it/s]


...

-2.2101164541132303 11.880921363830566


Epoch 49: 100%|██████████| 1/1 [00:00<00:00, 26.01it/s]


-2.021475935807809 11.253496170043945


Epoch 50: 100%|██████████| 1/1 [00:00<00:00, 24.18it/s]

-2.181490068655195 12.423982620239258


In [32]:
label_list = []
predictions_list = []

with tqdm(dataloader) as tq, torch.no_grad():
    for step, (batch_graph, batch_label) in enumerate(tq):
        batch_graph, batch_label = next(iter(dataloader))
        labels = batch_label[target_type]
        logits = model(batch_graph, batch_graph.ndata["feat"])[target_type]
        label_list.append(labels.cpu().numpy())
        predictions_list.append(logits.cpu().numpy())


cat_labels = np.concatenate(label_list)
cat_preds = np.concatenate(predictions_list)

100%|██████████| 1/1 [00:00<00:00, 28.51it/s]


In [16]:
print(cat_labels.shape)
print(cat_preds.shape)

(1632, 1)
(1632, 1)


In [33]:
r2 = r2_score(cat_labels[:,0], cat_preds[:,0])
print(r2)

0.8445300518290597


In [34]:
r2 = r2_score(cat_labels[:,1], cat_preds[:,1])
print(r2)

0.6478962999038074


In [35]:
r2 = r2_score(cat_labels[:,2], cat_preds[:,2])
print(r2)

0.43115008510261954
